In [ ]:
!wget https://repo.anaconda.com/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!bash Anaconda3-5.1.0-Linux-x86_64.sh
!source ~/.bashrc
!conda install pytorch=0.4.1

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Robust_Adversarial_Learning

/content/drive/MyDrive/Robust_Adversarial_Learning


In [5]:
from __future__ import print_function
import os
import argparse
import torch.nn as nn
import torchvision
import numpy as np
import torch.nn.functional as F
from losses.trades import trades_loss
import copy
import torch
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from models.wideresnet import *
from models.resnet import *
from models.small_cnn import *

In [6]:
# Use more of these as complexity increases

class Data:
  def __init__(self, train_loader, test_loader, attack_loader):
    self.train_loader = train_loader
    self.test_loader = test_loader
    self.attack_loader = attack_loader

class Model:
  model = None
  def __init__(self, id):
    self.id = id

class Loss:
  def __init__(self, loss_fn, id=None):
    self.loss_fn = loss_fn
    self.id = id

class Configuration:
  def __init__(self, data, model, loss, attack, id):
    self.data = data
    self.model = model
    self.loss = loss
    self.attack = attack

    self.id = id

  def getConfig(self):
    return self.data, self.model, self.loss, self.attack

  def getId(self):
    return self.id

In [19]:
configurations = []

In [17]:
use_cuda = torch.cuda.is_available()
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
])
transform_attack = transforms.Compose([transforms.ToTensor(),])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, **kwargs)
testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, **kwargs)
attackset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_attack)
attack_loader = torch.utils.data.DataLoader(attackset, batch_size=200, shuffle=False, **kwargs)

cifar10_data = Data(train_loader, test_loader, attack_loader)

100%|██████████| 170498071/170498071 [00:13<00:00, 13102219.78it/s]


Extracting ../data/cifar-10-python.tar.gz to ../data
Files already downloaded and verified
Files already downloaded and verified


In [11]:
def pgd_attack(epsilon=0.031, num_steps=20, step_size=0.003):

  def _pgd_whitebox(model,
                  X,
                  y,
                  epsilon=epsilon,
                  num_steps=num_steps,
                  step_size=step_size):
    out = model(X)
    err = (out.data.max(1)[1] != y.data).float().sum()
    X_pgd = Variable(X.data, requires_grad=True)
    # if args.random:
    #     random_noise = torch.FloatTensor(*X_pgd.shape).uniform_(-epsilon, epsilon).to(device)
    #     X_pgd = Variable(X_pgd.data + random_noise, requires_grad=True)

    for _ in range(num_steps):
        opt = optim.SGD([X_pgd], lr=1e-3)
        opt.zero_grad()

        with torch.enable_grad():
            loss = nn.CrossEntropyLoss()(model(X_pgd), y)
        loss.backward()
        eta = step_size * X_pgd.grad.data.sign()
        X_pgd = Variable(X_pgd.data + eta, requires_grad=True)
        eta = torch.clamp(X_pgd.data - X.data, -epsilon, epsilon)
        X_pgd = Variable(X.data + eta, requires_grad=True)
        X_pgd = Variable(torch.clamp(X_pgd, 0, 1.0), requires_grad=True)
    err_pgd = (model(X_pgd).data.max(1)[1] != y.data).float().sum()
    print('err pgd (white-box): ', err_pgd)
    return err_pgd

  return _pgd_whitebox

In [18]:
### Add Configurations

def general_trades_loss_fn(beta=6.0, epsilon=0.3, step_size=0.007, num_steps=10):
  def trades_loss_fn(model, data, target, optimizer):
    return trades_loss(model=model, x_natural=data, y=target, optimizer=optimizer, step_size=step_size,
                      epsilon=epsilon, perturb_steps=num_steps, beta=beta, distance='l_inf')
  return trades_loss_fn

for beta in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
  id = f'CIFAR10:WIDERESNET:TRADES_LOSS:BETA={beta}'
  wide_resnet = WideResNet().to(device)
  trades_loss_beta = Loss(general_trades_loss_fn(beta=beta), id)
  config1 = Configuration(cifar10_data, wide_resnet, trades_loss_beta, pgd_attack(), trades_loss_beta.id)
  configurations.append(config1)

# Ideally, all we have to do is just add our loss with same configs and run the whole thing

In [14]:
training_loss = {}
natural_accuracy = {}
robustness_accuracy = {}

In [21]:
def train(model, data_loader, loss, epochs=1):

  optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
  # TODO: We can move the optimizer to a field of Loss object

  model.to(device)

  for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for batch_idx, (data, target) in enumerate(data_loader):
          data, target = data.to(device), target.to(device)
          optimizer.zero_grad()
          l = loss.loss_fn(model, data, target, optimizer)
          l.backward()
          optimizer.step()
          total_loss += l.item()

    torch.save(model.state_dict(), os.path.join("weights", loss.id + str(epoch) + ".pt"))
    torch.save(optimizer.state_dict(), os.path.join("optimizers", loss.id + str(epoch) + ".tar"))

  return total_loss

def accuracy(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in data_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return 100. * correct / total

def eval_attack(model, attack, data_loader):

    model.eval()
    robust_err_total = 0
    natural_err_total = 0

    for data, target in data_loader:
        data, target = data.to(device), target.to(device)

        X, y = Variable(data, requires_grad=True), Variable(target)
        err_robust = attack(model, X, y)
        robust_err_total += err_robust
        # natural_err_total += err_natural
    return robust_err_total

In [ ]:
for (c, configuration) in enumerate(configurations):
  data, model, loss, attack = configuration.getConfig()

  training_loss[configuration.getId()] = train(model, data.train_loader, loss)
  natural_accuracy[configuration.getId()] = accuracy(model, data.test_loader)
  robustness_accuracy[configuration.getId()] = eval_attack(model, attack, data.attack_loader)

In [ ]:
print(training_loss)
print(natural_accuracy)
print(robustness_accuracy)

# TODO: Save this stuff to file